In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


from __future__ import absolute_import
from __future__ import print_function
from keras.utils import np_utils # For y values

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory



import os
print(os.listdir("../input"))
import pandas as pd
import numpy as np


# For plotting
%matplotlib inline
import seaborn as sns
# For Keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

In [ ]:
os.listdir("../input/mnist-in-csv")

In [ ]:
train = pd.read_csv("../input/mnist-in-csv/mnist_train.csv")

In [ ]:
train.head()

In [ ]:
#Remove the first column from the data, as it is the label and put the rest in X
X = train.iloc[:, 1:].values
#Remove everything except the first column from the data, as it is the label and put it in y
y = train.iloc[:, :1].values

In [ ]:
# The data does not load as a 28X28 image so there is no way to view it without reshaping it
X.shape

In [ ]:
x_train = X.reshape(-1,28, 28)
#x_train = X.reshape(42000,28, 28)

In [ ]:
x_train.shape

In [ ]:
#x_train[1:2]

In [ ]:
import matplotlib.pyplot as plt
#plot the first image in the dataset
plt.imshow(x_train[102])

In [ ]:
#set input to the shape of one X value
dimof_input = X.shape[1]

# Set y categorical
dimof_output = int(np.max(y)+1)
Y = np_utils.to_categorical(y, dimof_output)


In [ ]:
#x_train = X.reshape(-1,28, 28,1)

In [ ]:
Y[0]

In [ ]:
y[0]

# Below is a example of a simple MLP model

In [ ]:
mlp_model = Sequential()
mlp_model.add(Dense(16, input_dim=dimof_input, kernel_initializer='uniform', activation='relu'))
mlp_model.add(Dense(16,kernel_initializer='uniform', activation='relu'))
mlp_model.add(Dense(dimof_output, kernel_initializer='uniform', activation='softmax'))
mlp_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Here are a list of things you can play around with in the model above

#mlp_model.add(Dense(16, input_dim=dimof_input, kernel_initializer='uniform', activation='sigmoid'))

#Glorot normal initializer, also called Xavier normal initializer.
#mlp_model.add(Dense(16, input_dim=dimof_input, kernel_initializer='glorot_normal', activation='relu'))


#mlp_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#mlp_model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
#mlp_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [ ]:
mlp_model.summary()

In [ ]:
fit1 = mlp_model.fit(
    X, Y,
    validation_split=0.2,
    batch_size=500, epochs=5, verbose=1)

In [ ]:
def viz(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Valid'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Valid'], loc='upper left')
    plt.show()

In [ ]:
viz(fit1)
#fit1.history

# To combat over fitting some methods that can be used

In [ ]:
from keras.callbacks import EarlyStopping
ES = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1, mode='auto', baseline=None, restore_best_weights=False)

In [ ]:
fit1 = mlp_model.fit(
    X, Y,
    validation_split=0.2,
    batch_size=500, epochs=20, verbose=1, callbacks = [ES])

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
CP = ModelCheckpoint("model_{epoch:03d}.hdf5", monitor='loss', verbose=1,
    save_best_only=False, mode='auto', period=1)

In [ ]:
fit1 = mlp_model.fit(
    X, Y,
    validation_split=0.2,
    batch_size=500, epochs=5, verbose=1, callbacks = [CP])

In [ ]:
#So you can clear your output
def clear_out_of_weights():
    for i in os.listdir("../working"):
        if 'hdf5' in i:
            os.remove("../working/"+i)

In [ ]:
#clear_out_of_weights()

In [ ]:
load the weights of the model you want
#mlp_model.load_weights("../working/model_001.hdf5")

# Extra Material for trying different variations.

In [ ]:
#Play around with different optimizer settings.

from keras import optimizers
minimizer = optimizers.Adam(lr=0.00001)# if this doesnt work replac lr with leanring_rate

#SGD has paramters learning rate and momentum
#minimizer = optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False)


#The beta's are similar to momentum which is used in SGD
#minimizer = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)



mlp_model = Sequential()
mlp_model.add(Dense(16, input_dim=dimof_input, kernel_initializer='uniform', activation='relu'))
mlp_model.add(Dense(16,kernel_initializer='uniform', activation='relu'))
mlp_model.add(Dense(dimof_output, kernel_initializer='uniform', activation='softmax'))
mlp_model.compile(loss='categorical_crossentropy', optimizer=minimizer, metrics=['accuracy'])

In [ ]:
#Play around with different regulizers settings.

from keras.regularizers import l1,l2
from keras.layers import Activation

mlp_model = Sequential()
mlp_model.add(Dense(16, input_dim=dimof_input, kernel_initializer='uniform', activation='relu', kernel_regularizer=l1(0.001)))
mlp_model.add(Dense(16,kernel_initializer='uniform', activation='relu'))
mlp_model.add(Dense(dimof_output, kernel_initializer='uniform', activation='softmax'))
mlp_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
fit2 = mlp_model.fit(
    X, Y,
    validation_split=0.2,
    batch_size=500, epochs=7, verbose=1)

In [ ]:
viz(fit2)

In [ ]:
#Play with different dropout rates

from keras.regularizers import l1,l2
from keras.layers import Activation, Dropout

mlp_model = Sequential()
mlp_model.add(Dense(16, input_dim=dimof_input, kernel_initializer='uniform', activation='relu'))
mlp_model.add(Dropout(0.3))
mlp_model.add(Dense(16,kernel_initializer='uniform', activation='relu'))
mlp_model.add(Dense(dimof_output, kernel_initializer='uniform', activation='softmax'))
mlp_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
fit3 = mlp_model.fit(
    X, Y,
    validation_split=0.2,
    batch_size=500, epochs=7, verbose=1)

# CNN Model

In [ ]:
from keras import regularizers

cnn_model = Sequential()
cnn_model.add(Conv2D(input_shape = (28,28,1),
                 filters = 2, kernel_size = (5,5),padding = 'Same',
                 kernel_regularizer=regularizers.l2(0.01),
                 activation ='relu', strides = 3))

cnn_model.add(MaxPool2D(pool_size=(2,2)))
cnn_model.add(BatchNormalization())
cnn_model.add(Dropout(0.25))

cnn_model.add(Conv2D(filters = 4, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', strides = 3))
cnn_model.add(MaxPool2D(pool_size=(2,2)))
cnn_model.add(BatchNormalization())
cnn_model.add(Dropout(0.25))

cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation = "relu"))
cnn_model.add(Dense(10, activation = "softmax"))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
fit4 = cnn_model.fit(
    X.reshape(-1,28,28,1), Y,
    validation_split=0.2,
    batch_size=500, epochs=7, verbose=1)